In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [ ]:
class ShuffleLinear(nn.Module):
    def __init__(self, in_features, out_features):
        super(ShuffleLinear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.W = nn.Parameter(torch.Tensor(out_features, in_features))
        self.b = nn.Parameter(torch.Tensor(out_features))
        self.init_parameters()

    def init_parameters(self):
        nn.init.normal_(self.W, std=0.1)
        nn.init.constant_(self.b, 0)

    def forward(self, x):
        # Shuffle the input tensor and the layer kernel randomly
        indices = torch.randperm(x.size()[-1])
        shuffled_x = x[indices]
        shuffled_weight = self.W[:, indices]
        print(f"Size of input tensor: {x.size()}\nSize of W tensor {self.W.size()}")
        print(f"Input tensor: {x}\nWeight tensor: {self.W}")
        print(f"Shuffled indices: {indices}")
        print(f"Shuffled input: {shuffled_x}\nShuffled weighs: {shuffled_weight}")

        # Perform the matrix multiplication
        output = torch.matmul(shuffled_x, shuffled_weight.t())
        output_not_shuffled = torch.matmul(x, self.W.t())
        
        print(f"Shuffled output: {output}")
        print(f"output: {output_not_shuffled}")
        output = output + self.b
        return output
    

In [ ]:
inputX = torch.Tensor(np.arange(1, 4, dtype=float))
print(inputX)

In [ ]:
shuffle = ShuffleLinear(3, 4)
print(shuffle.forward(inputX))